In [32]:
import pandas as pd
import os
import datetime
import shutil
from PIL import Image

In [33]:
df = pd.read_csv("../../labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)
df

C:\Users\elise\AppData\Local\Temp\ipykernel_28956\1146341607.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)


,image_name,x_1,y_1,x_2,y_2
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262
...,...,...,...,...,...
289217,img/Paisley_Print_Babydoll_Dress/img_00000050.jpg,1,1,300,300
289218,img/Paisley_Print_Babydoll_Dress/img_00000051.jpg,14,58,225,277
289219,img/Paisley_Print_Babydoll_Dress/img_00000052.jpg,18,41,149,230
289220,img/Paisley_Print_Babydoll_Dress/img_00000053.jpg,75,47,220,300


In [34]:
category_df = pd.read_csv("../../labels/Anno_coarse/categories.txt", delimiter=",", header=0)
category_df

,category_name,category_type
0,Anorak,0
1,Blazer,1
2,Blouse,2
3,Bomber,3
4,Button-Down,4
5,Cardigan,5
6,Flannel,6
7,Halter,7
8,Henley,8
9,Hoodie,9


In [35]:
ttv = pd.read_csv("../../labels/Eval/list_eval_partition.txt", delim_whitespace=True, header=1)
'''for row in ttv.itertuples():
    if row.evaluation_status == 'test':
        ttv.at[row.Index, 'evaluation_status'] = 'train'''
ttv

C:\Users\elise\AppData\Local\Temp\ipykernel_28956\3654858314.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ttv = pd.read_csv("labels/Eval/list_eval_partition.txt", delim_whitespace=True, header=1)


,image_name,evaluation_status
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,train
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,train
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,val
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,train
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,test
...,...,...
289217,img/Paisley_Print_Babydoll_Dress/img_00000050.jpg,train
289218,img/Paisley_Print_Babydoll_Dress/img_00000051.jpg,train
289219,img/Paisley_Print_Babydoll_Dress/img_00000052.jpg,train
289220,img/Paisley_Print_Babydoll_Dress/img_00000053.jpg,train


In [36]:
df = df.merge(ttv, how='inner', on='image_name')
df

,image_name,x_1,y_1,x_2,y_2,evaluation_status
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273,train
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161,train
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200,val
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182,train
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262,test
...,...,...,...,...,...,...
289217,img/Paisley_Print_Babydoll_Dress/img_00000050.jpg,1,1,300,300,train
289218,img/Paisley_Print_Babydoll_Dress/img_00000051.jpg,14,58,225,277,train
289219,img/Paisley_Print_Babydoll_Dress/img_00000052.jpg,18,41,149,230,train
289220,img/Paisley_Print_Babydoll_Dress/img_00000053.jpg,75,47,220,300,train


In [37]:
def corr_positions(df, cat_num):
    '''Corriger les positions des boîtes englobantes pour les coordonnées des centres et les dimensions des boîtes englobantes
    df : DataFrame des positions des boîtes englobantes
    cat_num : int
    return : DataFrame'''
    
    corr_data = []   
    
    for row in df.itertuples():
        img_path = 'data/' + row.image_name
        with open(img_path, 'rb') as img_file:
            image = Image.open(img_file)
            img_width, img_height = image.size
        image = row.image_name.replace('img/', '')
        img_name = image.replace('/', '_')
        width = (row.x_2 - row.x_1) 
        height = (row.y_2 - row.y_1)
        x_center = ((width/2) + row.x_1) / img_width
        y_center = ((height/2) + row.y_1) /img_height
        width = width / img_width
        height = height / img_height
        corr_data.append({'image_name': img_name, 'category_type': cat_num, 'x_center': x_center, 'y_center': y_center, 'width': width, 'height': height, 'status': row.evaluation_status}) 
    corr_df = pd.DataFrame(corr_data)
    return corr_df

In [38]:
def find_category(df, category_name):
    '''Trouver la catégorie de vêtement dans le DataFrame
    df : DataFrame des catégories
    category_name : str
    return : int'''
    
    return df[df['category_name'].str.contains(category_name)].category_type.values[0]

Tests : 

Fonctions pour créer le dossier utilisé pour l'entraînement : copier les images et modifier les labels nécessaires :

In [39]:
def creer_fichier(chemin_fichier):
    '''Créer les dossier et fichiers nécessaires pour le fichier txt ou les images
    chemin_fichier : str'''

    dossier = os.path.dirname(chemin_fichier)
    if not os.path.exists(dossier):
        os.makedirs(dossier)

In [40]:
def modification_date(chemin_fichier):
    '''Trouver la date de la dernière modification d'un fichier
    chemin_fichier : str
    return : datetime'''
    
    modification_timestamp = os.path.getmtime(chemin_fichier)
    modification_date = datetime.datetime.fromtimestamp(modification_timestamp)
    return modification_date

In [41]:
def chemins(row):
    '''Mettre en forme le chemin du fichier txt depuis le DataFrame des positions des boîtes englobantes (qui possède le nom des images)
    row : Series
    return : str'''
    test_train_val = ttv.iloc[row.Index].evaluation_status 
    chemin_img = 'otherdata5/images/' + test_train_val + '/' + row.image_name
    chemin_label = 'otherdata5/labels/' + test_train_val + '/' + row.image_name.replace('.jpg', '') + '.txt'
    return chemin_img, chemin_label

In [42]:
def copie_images(df, row, chemin_img_arrivee):
    '''Copier les images dans le dossier de destination des images et labels à utiliser pour l'entraînement
    df : DataFrame des positions des boîtes englobantes
    category_df : DataFrame des catégories
    data_folder : str'''
    
    chemin_img_depart = 'data/' + df.iloc[row.Index].image_name
    shutil.copy2(chemin_img_depart, chemin_img_arrivee)

In [43]:
def creer_txt(df, row, chemin_fichier):
    '''Créer un fichier txt avec les coordonnées des boîtes englobantes
    df : DataFrame des bonnes coordonnées des boîtes englobantes (cat, x_center, y_center, width, height)
    row : Series
    chemin_fichier : str'''

    ligne = df.iloc[row.Index]
    ligne = ligne.drop(['image_name', 'status'])
    ligne_concatenee = ' '.join(map(str, ligne))
    with open(chemin_fichier, 'a+') as fichier:
        fichier.seek(0)  # Retourner au début du fichier
        contenu_existant = fichier.read()  # Lire le contenu existant du fichier
        if modification_date(chemin_fichier) < (datetime.datetime.now() - datetime.timedelta(seconds=30)):  # Si le fichier a été modifié pour la dernière fois il y a plus de 30 secondes, le vider
            fichier.truncate(0)
        elif contenu_existant:  # Sinon si le fichier n'est pas vide ajouter une nouvelle ligne
            fichier.write('\n')  # Cas utile lorsqu'il y a plusieurs vetements sur la meme image
        fichier.write(ligne_concatenee)

In [44]:
def prepare_data(df, category_df, category):
    '''Scinder les labels des boîtes englobantes pour chaque catégorie de vêtement
    Il faut un label (fichier txt) par image
    df : DataFrame des positions des boîtes englobantes
    category_df : DataFrame des catégories
    category : str'''

    cat = find_category(category_df, category)
    df_cat = df[df['image_name'].str.contains(category+ '/')]
    df_cat = df_cat.drop(df_cat[df_cat['image_name'].str.contains('Cr&egrave')].index, axis=0) #Le bug incompréhensible
    df_corr = corr_positions(df_cat, cat)
    for row in df_corr.itertuples():
        chemin_img, chemin_label = chemins(row)
        creer_fichier(chemin_label)
        creer_fichier(chemin_img)
        copie_images(df_cat, row, chemin_img)
        creer_txt(df_corr, row, chemin_label)

Préparer tout le dataset pour l'entraînement:

In [45]:
def prepare_data_toutes_categories(df, category_df):
    '''Scinder les labels des boîtes englobantes pour toutes les catégories de vêtements
    df : DataFrame des positions des boîtes englobantes'''
    
    for row in category_df.itertuples():
        prepare_data(df, category_df, row.category_name)

In [46]:
def choose_cat(df, category_df):
    cat_df = pd.DataFrame()
    for cat in category_df.itertuples():
        temp_df = df[df['image_name'].str.contains(cat.category_name + '/')]
        if temp_df[temp_df.columns[0]].count() > 400:
            print(cat.category_name, temp_df[temp_df.columns[0]].count())
            cat_df = pd.concat([cat_df, temp_df])
    return cat_df

In [47]:
new_df = choose_cat(df, category_df)

Blazer 7495
Blouse 24557
Cardigan 13311
Henley 716
Hoodie 4048
Jacket 10467
Jersey 748
Parka 676
Poncho 791
Sweater 13123
Tank 15429
Tee 36887
Top 10078
Chinos 527
Culottes 486
Cutoffs 1669
Jeans 7076
Jeggings 594
Joggers 4416
Leggings 5013
Shorts 19666
Skirt 14773
Sweatpants 3048
Sweatshorts 1106
Coat 2120
Dress 72158
Jumpsuit 6153
Kimono 2294
Romper 7408


In [48]:
new_df

,image_name,x_1,y_1,x_2,y_2,evaluation_status
750,img/Single-Button_Blazer/img_00000001.jpg,30,61,266,294,train
751,img/Single-Button_Blazer/img_00000002.jpg,14,74,212,300,val
752,img/Single-Button_Blazer/img_00000003.jpg,33,15,212,261,train
753,img/Single-Button_Blazer/img_00000004.jpg,41,62,213,281,train
754,img/Single-Button_Blazer/img_00000005.jpg,1,33,133,248,train
...,...,...,...,...,...,...
288782,img/Cutout_Back_Textured_Romper/img_00000030.jpg,88,25,249,300,val
288783,img/Cutout_Back_Textured_Romper/img_00000031.jpg,126,43,217,181,train
288784,img/Cutout_Back_Textured_Romper/img_00000032.jpg,38,65,195,300,val
288785,img/Cutout_Back_Textured_Romper/img_00000033.jpg,37,46,179,250,train


In [49]:
prepare_data_toutes_categories(new_df, category_df)

Tests :

In [50]:
#prepare_data(df.head(100), category_df, 'Blouse')

Création du fichier de configuration YAML pour YOLOv8 :

In [51]:
def create_yaml_file(yaml_file, category_df):
    with open(yaml_file, 'w') as file:
        file.write('path: ' + os.getcwd() + '\\otherdata5\n')
        file.write('train: images\\train\n')
        file.write('val: images\\val\n')
        file.write('names:\n')
        for row in category_df.itertuples():
            file.write('    ' + str(row.category_type) + ': ' + row.category_name + '\n')

In [52]:
create_yaml_file('../config.yaml', category_df)